In [77]:
"""****************************INSTRUMENT SETUP****************************"""
import csv
import socket
from KeithleyConnect import instrument_connect,instrument_query,instrument_write, instrument_disconnect


"""***********************************************************************"""
"""*********MAKE SURE THE KEITHLEY IS SET TO SCPI TO COMMUNITCATE*********"""
"""***********************************************************************"""


"""****************************INSTRUMENT CONNECTION****************************"""
#! Go into Windows and set the Ethernet connection to manual 
# define the instrament's IP address. the port is always 5025 for LAN connection.
ip_address = "169.254.233.64"
my_port = 5025

# establish connection to the LAN socket. initialize and connect to the Keithley
socket.socket().close()        # Close the connection if one already exists 
data_socket = socket.socket()  # Establish a TCP/IP socket object
instrument_connect(data_socket, ip_address, my_port, 10000, 0, 1) 


"""***************INSTRUMENT SET UP***************"""
# reset the device
instrument_write(data_socket, "*RST")

# define the measurement channels
channel_names = input()
channel_count = len(channel_names.split(','))
channel_names = str(channel_names)

# define the scan list, set scan count to infinite, and channel delay of 75 um
instrument_write(data_socket, ":ROUTe:SCAN:CRE (@{0})".format(channel_names)) # Create channels
instrument_write(data_socket, ":ROUTe:SCAN:COUN:SCAN 0") # Infinite scan count
instrument_write(data_socket, ":ROUTe:DEL 0.0002, (@{0})".format(channel_names)) # Channel delay
instrument_write(data_socket, ":ROUTe:SCAN:INT .05") # Scan to scan interval 




# choose the buffer and assign all data to the buffer after clearing it
instrument_write(data_socket, "TRACe:CLEar") 
instrument_write(data_socket, ":TRAC:FILL:MODE CONT, \"defbuffer1\"")
instrument_write(data_socket, ":ROUT:SCAN:BUFF \"defbuffer1\"")

# # define the channel functions
instrument_write(data_socket, "FUNC 'VOLT:DC', (@{0})".format(channel_names)) 

# # define channel parameters such as range to 1 volt, autozero off, and line sync on
instrument_write(data_socket, "VOLT:DC:RANG 1, (@{0})".format(channel_names)) # Range
instrument_write(data_socket, "VOLT:DC:AZER OFF, (@{0})".format(channel_names)) # Auto-zero off
instrument_write(data_socket, "VOLT:DC:LINE:SYNC ON, (@{0})".format(channel_names)) # Line sync on 
instrument_write(data_socket, "VOLT:DC:DEL:AUTO ON, (@{0})".format(channel_names)) # Auto-delay off
instrument_write(data_socket, ":SENS:VOLT:DC:NPLC 1, (@{0})".format(channel_names)) # NPLC
instrument_write(data_socket, "VOLT:DC:INP MOHM10, (@{0})".format(channel_names)) # Input impedance




# enable the graph to plot the data
instrument_write(data_socket, ":DISP:SCR HOME")
instrument_write(data_socket, "DISP:WATC:CHAN (@108)") 
instrument_write(data_socket, ":DISP:SCR GRAP")



KEITHLEY INSTRUMENTS,MODEL DAQ6510,04376148,1.7.12b



In [80]:
"""****************************DATA COLLECTION****************************"""
# begin data collection
instrument_write(data_socket,"INIT")

# press enter to stop data collection. we need the instrument query to be an 
# integer value of the channels. if not we get error 1115 
input('') == ()
while 1 == 1:
    if int(instrument_query(data_socket, "TRACe:ACTual:END? \"defbuffer1\"", 16).rstrip()) % channel_count == 0:       
        instrument_write(data_socket,"ABORT")
        break
    
# identify how many readings there are
readings_count = int(instrument_query(data_socket, "TRACe:ACTual?", 16).rstrip())

# preallocate counters and a Data list 
start_index = 1
end_index = channel_count
accumulated_readings = 0
Data = []

# read the buffer and place measurements into the 
while accumulated_readings < readings_count:
    Data.append(instrument_query(data_socket, "TRACe:DATA? {0}, {1}, \"defbuffer1\", REL, READ".format(start_index, end_index), 128).split(','))
    start_index += channel_count
    end_index += channel_count
    accumulated_readings += channel_count

# Notify the user of completion and the data streaming rate achieved. 
print("done")

done


In [1]:
"""****************************DATA EXPORT****************************"""
# define file name and its placement
file_name = 'P6_S1_F300mHz_mass100g'
directory_name = 'C:\\Users\\facci\\Desktop\\Kiethley Programming\\Data\\'

# make sure you changed the file name and let the program export the csv
if input('Did you change the file name') == 'yes':
    newFile = open('{0} {1}.csv'.format(directory_name, file_name),'w',newline='')
    newWriter = csv.writer(newFile, dialect='excel')
    for i in range(len(Data)):
        newWriter.writerow(Data[i])
    newFile.close()
else:
    print("Change the file name, dumbass")

print('Done')

Change the file name, dumbass
Done
